In [1]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
#sql牛逼
import MySQLdb
#zz牛逼教我json
import json
#你看看你个菜鸡用多久
import datetime
#系统牛逼
import os
#历史记录？
import time
#正则表达式天下第一
import re

In [2]:
def start():
    #万事开头难
    global conn,curs
    conn = MySQLdb.connect("127.0.0.1", "root", "nideshengri", "nidekuzi", charset='utf8' )
    # 使用cursor()方法获取操作游标 
    curs = conn.cursor()

In [3]:
#遍历并读取所有文件
def TransAndReadFile(Route):
    global files,alldata
    files=[]
    alldata=[]
    for root, dirs, files in os.walk(r'%s'%Route):
        pass
    #拜拜了您内.json   
    for i in range(len(files)):
        files[i]=files[i][:-5]
    #至此,files列表中存放了路径中的所有文件名
    #print(files[0])
    for mtag in files:
        fp=open(r"%s\\%s.json"%(Route,mtag),"rt", encoding='utf-8')
        file=[]
        for items in fp.readlines():
            for item in json.loads(items):
                file.append(item)
        alldata.append(file)
        fp.close()
    #效果拔群
    #至此，alldata列表中的每项都存放了各个文件的所有内容
    #print(alldata[4][7])

In [42]:
#插两个新表啊
def CreateTable(mtag):
    sql='''CREATE TABLE `nidekuzi`.`%s` ( `SteamId` INT(100)  , `Name` varchar(200)	 , `Platforms` TEXT , `Pic_url` TEXT  , `CurrentPrice` TEXT ,`OriginalPrice` TEXT  , `Cut` TEXT ,`LowestPrice` TEXT  ,INDEX `Name` (`Name`) , PRIMARY KEY (`SteamId`)) ENGINE = MyISAM'''%mtag
    #try:
    curs.execute(sql)
    #except:
        #conn.rollback()
    conn.commit()

    sql='''CREATE TABLE `nidekuzi`.`%sUpdata` ( `SteamId` INT(100)  ,`Price` TEXT  ,`UpdataTime` TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP , INDEX (`SteamId`)) ENGINE = MyISAM'''%mtag
    #try:
    curs.execute(sql)
    #except:
        #conn.rollback()
    conn.commit()

In [5]:
#插入游戏tag
def inserttag(idn,tags):
    for tag in tags.split(','):
        sql="""INSERT INTO `nidekuzi`.`id_tag`(SteamId,Tag) VALUES (%d,"%s")"""%(idn,tag.lstrip())
        #try:
        curs.execute(sql)
        #except:
            #conn.rollback()
        conn.commit()

In [24]:
#插入游戏的函数
def insertgame(idn,name,price,tags):
    if len(price)==0:
        return      #价格为空意味着游戏已经下架
    name=filt_emo(name)     #为什么有人在游戏名里加emoji？
    name=transferContent(name)      #转义真是太难了太难了...
    price[-1]=filt(price[-1])
    price[0]=filt(price[0])
    if float(price[-1][:-1])==0:
        cut='0%'
    else:
        cut=str(int((float(price[-1][:-1])-float(price[0][:-1]))/float(price[-1][:-1])*100))
        cut=cut+'%'
    sql = """INSERT INTO `%s`(`SteamId`, `Name`, `CurrentPrice`, `LowestPrice`, `OriginalPrice` , `Cut`)
             VALUES ('%d','%s','%s','%s','%s','%s')"""%(mtag,idn,name,filt(price[0]),filt(price[0]),filt(price[-1]),cut)
    try:
        curs.execute(sql)
    except:
        conn.rollback()
    conn.commit()
    inserttag(idn,tags)

In [7]:
#转义NMSL，代码我抄的https://blog.csdn.net/lengye7/article/details/79916685
def transferContent(content):
        if content is None:
            return None
        else:
            string = ""
            for c in content:
                if c == '"':
                    string += '\\\"'
                elif c == "'":
                    string += "\\\'"
                elif c == "\\":
                    string += "\\\\"
                else:
                    string += c
            return string

In [8]:
#求求你们不要在游戏名里加emoji了！！
def filt_emo(name):
    restr=''
    try:
        co = re.compile(u'[\U00010000-\U0010ffff]')
    except re.error:
        co = re.compile(u'[\uD800-\uDBFF][\uDC00-\uDFFF]')
    return co.sub(restr, name)

In [21]:
#金钱格式一定要统一啊....
def filt(price):
    price=re.findall("\d+\\.?\d*",price)
    if len(price)==0:
        return '0.00￥'
    else:
        price=float(price[0])
        return '%.2f￥'%price

In [10]:
#插入图片函数
def inspic(idn,url):
    sql='''UPDATE `%s` SET `Pic_url` = '%s' WHERE `%s`.`SteamId` = %d'''%(mtag,url,mtag,idn)
    #try:
    curs.execute(sql)
    #except:
        #conn.rollback()
    conn.commit()

In [40]:
#更新史低数据的函数
def updatalowest(mtag,idn,platforms,newest,cut,lowest,price):
    sql="SELECT * FROM `%s` WHERE SteamId=%d"%(mtag,idn)
    curs.execute(sql)
    olddata=curs.fetchone()
    if olddata==None:#意味着有新游戏或者游戏遗漏，那么爬的数据不全
        return
    if newest.isspace():
        newest=olddata[-4]
    if cut.isspace():
        cut='0%'
    if lowest.isspace():
        lowest=olddata[-1]
    if price.isspace():
        price=olddata[-3]
    newest=filt(newest)
    lowest=filt(lowest)
    price=filt(price)
    if newest != olddata[-4]:
        recording(mtag,idn,newest,time)
    sql='''UPDATE `%s` SET `Platforms` = '%s', `CurrentPrice` = '%s', `OriginalPrice` = '%s', `Cut` = '%s', `LowestPrice` = '%s' WHERE `%s`.`SteamId` = %d'''%(mtag,platforms,newest,price,cut,lowest,mtag,idn)
    #try:
    curs.execute(sql)
    #except:
        #conn.rollback()
    conn.commit()

In [12]:
#在updata表里留下证据
def recording(mtag,idn,price,time):
    sql = """INSERT INTO `%sUpdata`(`SteamId`, `Price`)
             VALUES ('%d','%s')"""%(mtag,idn,price)
    #try:
    curs.execute(sql)
    #except:
        #conn.rollback()
    conn.commit()

In [13]:
#通用更新函数
#四个参数为表名，id，需要更新的数据，更新数据的字段名
def updatasql(mtag,idn,data,dataname):
    sql='''UPDATE `%s` SET `%s` = '%s' WHERE `%s`.`SteamId` = %d'''%(mtag,dataname,data,mtag,idn)
    #try:
    curs.execute(sql)
    #except:
        #conn.rollback()
    conn.commit()

In [43]:
#主函数来了！！！！
#插入id，游戏名，价格标签
ks=datetime.datetime.now()
TransAndReadFile(r'C:\Users\Su\OneDrive\steam\bas')
start()
for file,mtag in zip(alldata,files):
    CreateTable(mtag)
    for item in file :
        insertgame(int(item['id']),item['name'],item['price'],item['tags'])
js=datetime.datetime.now()
print(js-ks)

0:00:53.708784


In [44]:
#插入图片地址，这段代码好像有点费时间
ks=datetime.datetime.now()
TransAndReadFile(r'C:\Users\Su\OneDrive\steam\pic')
start()
for file,mtag in zip(alldata,files):
    for item in file :
        updatasql(mtag,int(item['id']),item['picture'],'Pic_url')
js=datetime.datetime.now()
print(js-ks)

0:00:10.589207


In [45]:
#更新史低数据
ks=datetime.datetime.now()
TransAndReadFile(r'C:\Users\Su\OneDrive\steam\updata')
start()
app=0
for file,mtag in zip(alldata,files):
    for item in file :
        if item['id']=='app':
            app+=1
        else:
            updatalowest(mtag,int(item['id']),item['platforms'],item['newest'],item['cut'],item['lowest'],item['price'])
js=datetime.datetime.now()
print(js-ks)

0:00:21.326508


In [0]:
# 关闭连接
conn.close()

In [0]:
'''# SQL 删除语句
sql = "DELETE FROM games "
try:
   # 执行SQL语句
   curs.execute(sql)
   # 提交修改
   conn.commit()
except:
   # 发生错误时回滚
   conn.rollback()'''

In [0]:
#我是傻逼,我不会用json
'''#正则表达式天下第一
import re
allgames=[]
for largeitem in content:
    for item in largeitem.split("}"):
        if len(item)<5 :
            continue
        else:
            #item=re.findall('"(.*?)"',item)
            allgames.append(item)'''

In [0]:
#产品经理wyx一开始比较喜欢写0
'''#插入所有的标签
for item in allgames:
    for tag in item['tags'].split(','):
        zjzd(tag)'''